In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torchtext import data
import re 
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
label = pd.read_csv('label.csv') # Labelled Data provided
label['id'] = label['id'].apply(lambda x: x.split('-')[0]).astype(int)

/home/gwan/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to /home/gwan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
label[label.annotation == 1]

,id,text,annotation,label
1,1573658,ccu nursing progress note still feel alittle n...,1,1
4,1472567,sicu nsg note neuro ct scan head showed atroph...,1,1
7,561667,title chief complaint fever tachypnea hpi 73yo...,1,1
11,354817,altered mental status delirium assessment pati...,1,1
12,726477,87 female admitted er w resp distress family s...,1,1
...,...,...,...,...
519,2009786,progress note 7a 7p resp ra c e mild scr rr wn...,1,1
520,588510,title chief complaint dyspnea 24 hour events g...,1,1
523,42148,admission date 2118 8 10 discharge date 2118 8...,1,1
527,441775,chief complaint rigors 24 hour events 3 12 eve...,1,1


In [2]:
label.loc[label.annotation == 'rejected','annotation'] = 0.
label.loc[label.annotation != 0,'annotation'] = 1


In [3]:
from numpy import int64


label = label[['id','text','annotation']]
label['label'] = label['annotation'].astype(int64)

In [2]:
def cleanup_text(texts):
    text = texts
    en_stops = set(stopwords.words('english'))
    
    # Lower case
    text = text.lower()
     # remove punctuation
    text = re.sub('[^a-zA-Z0-9]', ' ', text)
    # remove multiple spaces
    text = re.sub(r' +', ' ', text)
    # remove newline
    text = re.sub(r'\n', ' ', text)
    # Remove stopwords
    filtered_words = [word for word in text.split(' ') if word not in stopwords.words('english')]
    return ' '.join(filtered_words)

In [5]:
label['text'] = label['text'].apply(cleanup_text)

# Split data
validation_percent = 0.20

# Split data into train / validate
train,test = train_test_split(label,test_size=validation_percent,random_state = 10)

In [6]:
label.text

0      nicu nursing progress note please refer flowsh...
1      ccu nursing progress note still feel alittle n...
2      title chief complaint declining mental status ...
3      neuro pt movements vigorous eyes open often re...
4      sicu nsg note neuro ct scan head showed atroph...
                             ...                        
527    chief complaint rigors 24 hour events 3 12 eve...
528    7p 7a micu progress note neuro pt remains mini...
529    chief complaint food stuck throat fevers cough...
530    pca note 0700 1900 1 resp 360 remains ra rr 30...
531    full code nkda 55 f morbid obesity htn recent ...
Name: text, Length: 532, dtype: object

In [11]:
train.to_csv('train.csv',index = False)
test.to_csv('test.csv',index =False)

In [8]:
train

,id,text,annotation,label
245,466229,social work covering first name8 namepattern2 ...,0.0,0
278,395732,attending physician name10 nameis 116 referral...,0.0,0
446,1879255,nrsg progress note 0700 1500 1 5o aremains amp...,0.0,0
60,1545160,nursing progress note resp pt received cpap ps...,1,1
217,1642224,npn micu mrs known lastname cont recover uti s...,1,1
...,...,...,...,...
369,1872014,npn 1900 0700 2 resp remains ra spells dsats i...,0.0,0
320,1486031,pmicu npn 7p 7a events overnoc included brief ...,1,1
527,441775,chief complaint rigors 24 hour events 3 12 eve...,1,1
125,4466,admission date 2169 11 21 discharge date 2169 ...,0.0,0


# Sentence data

In [4]:
sentence_data = pd.read_csv('sentence.csv')

In [ ]:
sentence_data.rename(columns={"target": "labels"})

In [8]:
sentence_data['sentence'] = sentence_data['sentence'].apply(cleanup_text)

# Split data
validation_percent = 0.20

# Split data into train / validate
train,test = train_test_split(sentence_data,test_size=validation_percent,random_state = 10)

In [31]:
for i,row in sentence_data.iterrows():
    if(row['target'] == 1):
        print(row['sentence'])
        print('end ...')
    # print(len(row['sentence']))

She has + bowel sounds but no stool as yet.House staff waiting on placing a FT for now as she was very uncooperative earlier.Her BS are improved on current SS and fixed dosing.
end ...
His day consists of going for a number of walks with
   intervals of [**Location (un) 308**] personality magazines, and playing his guitar.One
   way he has found for dealing with his anger is listing to a [**Doctor Last Name **] group
   from CA because they are as angry as he is.Most of his anger is
   directed at the negative state of the world.
end ...
 LS clear at apices to diminished at bases, RR 11-22, O2 sats 98-100% on 1.5 L n.c.

Neuro Pt alert and oriented x 3, pleasant and cooperative with care, asking appropriate questions regarding care, MAE.Pt OOB to commode with supervision, pt BP tol.
end ...
Discussed
that prognosis at this time is poor in regards to neurological
outcome.Per family, at baseline, patient was very active
(played golf, went dancing, went to the gym) and he would not
want t

In [34]:
sentence_data['target'].value_counts()

0    73
1    69
Name: target, dtype: int64